In [310]:
import numpy as np
import math

In [311]:
def sigmoid(z):
    return 1./(1+np.exp(-z))

In [312]:
def log_loss(t, y):
    return np.sum((-t*np.log(y) - (1 - t) * (np.log(1 - y))), keepdims=True).flatten()

def diff_log_loss(t, y):
    return np.sum(-t/y +(1-t)/(1-y), keepdims=True)

In [313]:
def logos(sample):
    sigmoid(sample.dot(w) + b)

In [314]:
class Layer: #слой реализован в виде класса
  
    
    def __init__(self, n_inp, n_out, lr=0.1):
        self.shape = (n_inp, n_out) #залетают входящие и выходящие значения
        self.lr = lr #шаг обучения
        self.w = np.zeros(self.shape, dtype=np.float32) #веса в начале заданы нулями
        self.b = np.zeros((1, n_out), dtype=np.float32) #смещения в начале заданы нулями
        self._clear_grads() #проход обычного градиентного спуска

    def _clear_grads(self): #градиентный спуск
        self.inp = None
        self.activations = None
        self.d_sigma = None
        self.d_w = None
        self.d_b = None

    def __call__(self, x): #вызов класса
        if len(x.shape) == 1: #если входящие значения одноразмерны - сменить размерность
            x = x.reshape(1, -1)
        self.inp = x #задаем входные значения
        self.activations = sigmoid(x.astype(np.float64).dot(self.w) + self.b) #запуск функции с функцией активации
        return self.activations

    def backward(self, grad): # обратный проход
        self.d_sigma = self.activations * (1 - self.activations) #производная функции ошибки 
        self.d_w = self.grad_w(grad)
        self.d_b = self.grad_b(grad)
        return self.grad_x(grad)

    def grad_w(self, grad):
        return grad * self.inp.T * self.d_sigma

    def grad_b(self, grad):
        return grad * self.d_sigma

    def grad_x(self, grad):
        return self.w.dot(grad) * self.d_sigma

    def step(self):
        self.w -= self.d_w * self.lr
        self.b -= self.d_b * self.lr
        self._clear_grads()
        #return(self.w, self.b)
    
    def logos(self,sample):
        return(sigmoid(sample.dot(self.w) + self.b))

In [315]:
l1 = Layer(5, 1)

In [316]:
l1.w

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)

In [317]:
mu_0 = np.random.normal(loc=(-1, -1), scale=(1, 1), size =(10,1)).reshape(10,)
mu_1 = np.random.normal(loc=(1, 1), scale=(1, 1), size =(10,1)).reshape(10,)

In [318]:
x = np.array([1., 2., 1., -1., -2.])

In [319]:
x.shape

(5,)

In [320]:
mu_0

array([-2.1437663 , -1.36419864, -0.64039143, -1.7815914 , -1.96387339,
       -0.39488693, -1.77022704,  0.06180425, -0.47279055, -0.77047019])

In [321]:
mu_1 

array([0.05959778, 0.69243402, 2.19719159, 1.47952505, 0.98433451,
       0.12939022, 0.29138256, 0.53457187, 1.2584286 , 1.68335228])

In [322]:
t = mu_1
t.shape

(10,)

In [323]:
y = l1(x)

In [324]:
y

array([[0.5]])

In [325]:
log_loss(t[0], y)

array([0.69314718])

In [326]:
d_y = diff_log_loss(t[0], y)

In [327]:
d_y

array([[1.76160889]])

In [328]:
for line in t:
    #print(line.reshape(1,).shape)
    q = 0
    for i in range(10):
        y = l1(x)
        d_y = diff_log_loss(line.reshape(1,), y)
        #print(d_y)
        l1.backward(d_y)
        l1.step()
    print(d_y)

[[0.57346932]]
[[-0.15843259]]
[[-1651325.48970317]]
[[-4.9380632e+08]]
[[24216945.48077714]]
[[108980.90231499]]
[[15.84361458]]
[[0.12978508]]
[[-22.43953864]]
[[-125039.78754773]]


In [329]:
y

array([[0.99999453]])

In [330]:
t2 = mu_0

In [331]:
for line in t2:
    #print(line.reshape(1,).shape)
    q = 0
    for i in range(10):
        y = l1(x)
        d_y = diff_log_loss(line.reshape(1,), y)
        #print(d_y)
        l1.backward(d_y)
        l1.step()
    print(d_y)

[[6828685.20724842]]
[[1.42520209e+14]]
[[3.46804018e+17]]
[[4.72645213e+26]]
[[7.18843806e+36]]
[[1.0855439e+39]]
[[1.57053384e+48]]
[[-2.35340342e+47]]
[[2.75878144e+50]]
[[3.25848728e+54]]


In [332]:
y

array([[2.36450268e-55]])

In [333]:
z = np.array([0.0609173])

In [334]:
log_loss(z, y)

array([7.66227622])

In [339]:
z = np.array([-0.06180425])

In [340]:
log_loss(z, y)

array([-7.77383822])